# JSON-LD Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) Resources to [JSON-LD](https://json-ld.org) and vice-versa. JSON-LD is a semantic-preserving JSON format allowing to provide identifiers and meaning to JSON keys and values by mean of an added '@context' object. Read the excellent [JSON-LD documentation](https://json-ld.org/learn.html) to learn more about this format.

A JSON-LD context can be assigned to a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) in three ways by order of priority:
1. directly set in the resource using the property `context`: e.g.`jane_resource.context="https://schema.org/docs/jsonldcontext.json"`
2. set in the `Model` section of the forge configuration file: e.g.

'''

Model:

  name: RdfModel
  
  origin: ...
  
  source: ...
  
  context:
  
    iri: "https://schema.org/docs/jsonldcontext.json"
    
'''    

3. set by the configured store. A store (such as BlueBrainNexus) can support JSON-LD and add a default JSON-LD context whenever one is not provided.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

# Imports

In [3]:
import json

In [4]:
from kgforge.core import Resource

In [5]:
def pp(x): 
    print(json.dumps(x, indent=4))

## Context

In [6]:
context = {
    "ex": "http://example.org/",
    "Person": "ex:Person",
    "Organization": "ex:Organization",
    "employer": "ex:employer",
    "name": "ex:name"
}

## Resource to JSON-LD

### context from the user

#### locally defined context

In [7]:
jane = Resource(context=context, type="Person", name="Jane Doe")

In [8]:
forge.as_json(jane)

{'type': 'Person', 'name': 'Jane Doe'}

In [9]:
pp(forge.as_jsonld(jane))

{
    "@context": {
        "ex": "http://example.org/",
        "Person": "ex:Person",
        "Organization": "ex:Organization",
        "employer": "ex:employer",
        "name": "ex:name"
    },
    "@type": "Person",
    "name": "Jane Doe"
}


In [10]:
pp(forge.as_jsonld(jane, form="expanded"))

{
    "@type": "http://example.org/Person",
    "http://example.org/name": "Jane Doe"
}


#### remote context from the web

In [11]:
jane = Resource(context="https://schema.org/docs/jsonldcontext.json", type="Person", name="Jane Doe")

In [12]:
print(jane)

{
    context: https://schema.org/docs/jsonldcontext.json
    type: Person
    name: Jane Doe
}


In [13]:
pp(forge.as_jsonld(jane))

{
    "@context": "https://schema.org/docs/jsonldcontext.json",
    "type": "Person",
    "name": "Jane Doe"
}


### context from the model

The configured model provides a default context that will be used to create resources that do not have context provided.

In [14]:
jane = Resource(type="Person", name="Jane Doe")

In [15]:
print(jane)

{
    type: Person
    name: Jane Doe
}


In [16]:
pp(forge.as_jsonld(jane))

{
    "@context": {
        "@vocab": "https://neuroshapes.org/",
        "AcquisitionAnnotation": {
            "@id": "nsg:AcquisitionAnnotation"
        },
        "Activity": {
            "@id": "prov:Activity"
        },
        "AffineLinearTransform": {
            "@id": "nsg:AffineLinearTransform"
        },
        "Agent": {
            "@id": "prov:Agent"
        },
        "Analysis": {
            "@id": "nsg:Analysis"
        },
        "AnalysisConfiguration": {
            "@id": "nsg:AnalysisConfiguration",
            "@type": "@id"
        },
        "AnalysisReport": {
            "@id": "nsg:AnalysisReport",
            "@type": "@id"
        },
        "AnalysisResult": {
            "@id": "nsg:AnalysisResult"
        },
        "AnnotatedSlice": {
            "@id": "nsg:AnnotatedSlice"
        },
        "Annotation": {
            "@id": "nsg:Annotation"
        },
        "AnnotationBody": {
            "@id": "nsg:AnnotationBody"
        },
        "Apical

In [17]:
pp(forge.as_jsonld(jane, form="expanded"))

{
    "@type": "http://schema.org/Person",
    "http://schema.org/name": "Jane Doe"
}


### context from the Store

It is possible to use a context that is available in the Store when configured.

In [18]:
jane = Resource(context="https://bbp.neuroshapes.org", type="Person", name="Jane Doe")

In [19]:
print(jane)

{
    context: https://bbp.neuroshapes.org
    type: Person
    name: Jane Doe
}


Since this context is not locally resolvable the json-ld conversion will fail.

In [20]:
pp(forge.as_jsonld(jane))

<action> _resource_context
<error> ValueError: https://bbp.neuroshapes.org is not resolvable

null


In [21]:
pp(forge.as_jsonld(jane, form="expanded"))

<action> _resource_context
<error> ValueError: https://bbp.neuroshapes.org is not resolvable

null


In [22]:
forge.register(jane)

<action> _register_one
<succeeded> False
<error> ValueError: https://bbp.neuroshapes.org is not resolvable


In [23]:
pp(forge.as_jsonld(jane, store_metadata=True))

<action> _resource_context
<error> ValueError: https://bbp.neuroshapes.org is not resolvable

null


## JSON-LD to Resource

In [24]:
john = Resource(context=context, type="Person", name="John Smith")

In [25]:
data = {
    "@context": context,
    "@type": "Person",
    "name": "John Smith",
}

In [26]:
resource = forge.from_jsonld(data)

In [27]:
resource == john

True